# [EBGAN](https://arxiv.org/pdf/1609.03126.pdf) [local-paper](http://localhost:8888/notebooks/Dropbox/Paper/1609.03126.pdf) [Yann LeCun's talk at NIPS 2016](https://www.youtube.com/watch?v=88nKI-qqWEo)  [code](https://github.com/wiseodd/generative-models/tree/master/GAN/ebgan) 

<img src="img/EBGAN - OBJECTIVE FUNCTIONAL.png"/>

<img src="img/EBGAN - USING AUTO-ENCODERS.png"/>

Code by
[buriburisuri](https://github.com/buriburisuri/ebgan)
[shekkizh](https://github.com/shekkizh/EBGAN.tensorflow)


Energy-Based Generative Adversarial Network
[Chunyuan Li](http://people.ee.duke.edu/~lcarin/Chunyuan3.25.2017.pdf) 
[local-slide](http://localhost:8888/notebooks/Dropbox/Paper/Chunyuan3.25.2017.pdf) 

inFERENCe
[Are Energy-Based GANs any more energy-based than normal GANs?](http://www.inference.vc/are-energy-based-gans-actually-energy-based/)

In [1]:
# https://github.com/wiseodd/generative-models/blob/master/GAN/ebgan/ebgan_tensorflow.py

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os


mb_size = 32
X_dim = 784
z_dim = 64
h_dim = 128
lr = 1e-3
m = 5

mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)


def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig


def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)


X = tf.placeholder(tf.float32, shape=[None, X_dim])
z = tf.placeholder(tf.float32, shape=[None, z_dim])

D_W1 = tf.Variable(xavier_init([X_dim, h_dim]))
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))
D_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
D_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

G_W1 = tf.Variable(xavier_init([z_dim, h_dim]))
G_b1 = tf.Variable(tf.zeros(shape=[h_dim]))
G_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]
theta_D = [D_W1, D_W2, D_b1, D_b2]


def sample_z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob


def discriminator(X):
    D_h1 = tf.nn.relu(tf.matmul(X, D_W1) + D_b1)
    X_recon = tf.matmul(D_h1, D_W2) + D_b2
    mse = tf.reduce_mean(tf.sqrt(tf.reduce_sum((X - X_recon)**2, 1)))
    return mse


G_sample = generator(z)

D_real = discriminator(X)
D_fake = discriminator(G_sample)

D_loss = D_real + tf.maximum(0., m - D_fake)
G_loss = D_fake

D_solver = tf.train.AdamOptimizer(learning_rate=lr).minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer(learning_rate=lr).minimize(G_loss, var_list=theta_G)

with tf.Session() as sess:
    tf.global_variables_initializer().run()

    if not os.path.exists('out/'):
        os.makedirs('out/')

    i = 0
    for it in range(1000000):
        X_mb, _ = mnist.train.next_batch(mb_size)
        z_mb = sample_z(mb_size, z_dim)
        _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, z: z_mb})
        _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={X: X_mb, z: sample_z(mb_size, z_dim)})
        if it % 1000 == 0:
            print('Iter: {}; D_loss: {:.4}; G_loss: {:.4}'.format(it, D_loss_curr, G_loss_curr))
            samples = sess.run(G_sample, feed_dict={z: sample_z(16, z_dim)})
            fig = plot(samples)
            plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
            i += 1
            plt.close(fig)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz
Iter: 0; D_loss: 14.78; G_loss: 21.04
Iter: 1000; D_loss: 5.427; G_loss: 5.811
Iter: 2000; D_loss: 5.823; G_loss: 4.817
Iter: 3000; D_loss: 5.645; G_loss: 5.455
Iter: 4000; D_loss: 5.185; G_loss: 5.239
Iter: 5000; D_loss: 5.115; G_loss: 4.72
Iter: 6000; D_loss: 5.456; G_loss: 5.101
Iter: 7000; D_loss: 5.504; G_loss: 5.242
Iter: 8000; D_loss: 5.337; G_loss: 5.017
Iter: 9000; D_loss: 5.341; G_loss: 5.518
Iter: 10000; D_loss: 5.405; G_loss: 4.893
Iter: 11000; D_loss: 5.206; G_loss: 4.838
Iter: 12000; D_loss: 5.079; G_loss:

KeyboardInterrupt: 